<a href="https://colab.research.google.com/github/williamfaraday123/SC2008/blob/main/SC2008_Lab4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LAB 4:  ANALZING NETWORK DATA LOG

### You are provided with the data file, in .csv format, in the working directory.  Write the program to extract the following informations.

In [1]:
!pip install ipwhois

In [2]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import plotly.express as px
from ipwhois import IPWhois
from google.colab import drive

# Given SFlow Format in Table 1
format = ['Type', 'sflow_agent_address', 'inputPort', 'outputPort',
               'src_MAC', 'dst_MAC', 'ethernet_type', 'in_vlan', 'out_vlan',
               'src_IP', 'dst_IP', 'IP_protocol', 'ip_tos', 'ip_ttl',
               'udp_src_port/tcp_src_port/icmp_type', 'udp_dst_port/tcp_dst_port/icmp_code', 'tcp_flags',
               'packet_size', 'IP_size', 'sampling_rate', '???']

# EXERCISE 4A: TOP TALKERS AND LISTENERS

### One of the most commonly used function in analyzing data log is finding out the IP address of the hosts that send out large amount of packet and hosts that receive large number of packets, usually know as TOP TALKERS and LISTENERS.  Based on the IP address we can obtained the organization who owns the IP address.

In [3]:
drive.mount('/content/drive')

log_df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Data_3.csv', header = None, names = format)
log_df.drop('???', axis=1, inplace = True) # Drop column with null values
log_df.head(10)

Mounted at /content/drive


,Type,sflow_agent_address,inputPort,outputPort,src_MAC,dst_MAC,ethernet_type,in_vlan,out_vlan,src_IP,dst_IP,IP_protocol,ip_tos,ip_ttl,udp_src_port/tcp_src_port/icmp_type,udp_dst_port/tcp_dst_port/icmp_code,tcp_flags,packet_size,IP_size,sampling_rate
0,FLOW,203.30.38.251,137,200,d404ff55fd4d,80711fc76001,0x0800,919,280,130.246.176.22,140.115.32.81,6,0x00,50,51216,23505,0x10,1518,1500,2048
1,FLOW,203.30.38.251,129,193,609c9f851b00,0031466b23cf,0x0800,11,919,155.69.160.32,64.233.188.128,6,0x00,56,23159,80,0x10,74,52,2048
2,FLOW,203.30.38.251,137,200,d404ff55fd4d,80711fc76001,0x0800,919,280,130.246.176.53,140.115.32.83,6,0x00,50,50041,20739,0x10,1518,1500,2048
3,FLOW,203.30.38.251,129,135,609c9f851b00,002688cd5fc7,0x0800,11,919,155.69.160.32,54.169.174.79,17,0x00,120,54241,26510,0x10,116,94,2048
4,FLOW,203.30.38.251,130,199,00239cd087c1,544b8cf9a7df,0x0800,919,600,137.132.228.15,193.62.192.8,6,0x00,56,53923,34262,0x10,70,52,2048
5,FLOW,203.30.38.251,129,135,609c9f851b00,002688cd5fc7,0x0800,11,919,155.69.160.32,54.255.221.151,17,0x00,120,55069,26509,0x10,113,91,2048
6,FLOW,203.30.38.251,130,199,00239cd087c1,544b8cf9a7df,0x0800,919,600,137.132.250.8,193.62.193.9,6,0x00,62,27419,80,0x10,58,40,2048
7,FLOW,203.30.38.251,137,200,d404ff55fd4d,80711fc76001,0x0800,919,280,193.61.196.206,140.110.147.170,6,0x00,240,443,43545,0x18,1518,1500,2048
8,FLOW,203.30.38.251,200,3,80711fc76001,00235ed9b680,0x0800,280,32,137.189.133.62,123.136.64.7,6,0x00,58,6345,34921,0x10,1342,1320,2048
9,FLOW,203.30.38.251,199,130,544b8cf9a7df,00239cd087c1,0x0800,600,919,193.62.192.8,137.132.228.15,6,0x08,55,34262,53923,0x10,1442,1420,2048


In [4]:
# Finding organisation given IP
def find_org(ip_addr):
    ip = IPWhois(ip_addr)
    result = ip.lookup_rdap()
    return result.get('network', {}).get('name')

In [5]:
# Top 5 unique IP addresses by number of packets sent
top_talkers = log_df['src_IP'].value_counts().nlargest(5).to_frame()
org = []
for ip_addr in top_talkers.index:
    org.append(find_org(ip_addr))
top_talkers['Organisation'] = org    # append org. data to the dataframe

top_talkers = top_talkers.reset_index().rename(columns = {'index':'IP Address',
                                                        'src_IP':'No. of Packets'})
top_talkers

,No. of Packets,count,Organisation
0,193.62.192.8,3041,EUR-BIO-INST
1,155.69.160.32,2975,NTUNET1
2,130.14.250.11,2604,NLM-ETHER
3,14.139.196.58,2452,NKN-IIT-GUW
4,140.112.8.139,2056,T-NTU.EDU.TW-NET


In [6]:
# Top 5 unique IP addresses by number of packets received
top_listeners = log_df['dst_IP'].value_counts().nlargest(5).to_frame()
org = []
for ip_addr in top_listeners.index:
    org.append(find_org(ip_addr))
top_listeners['Organisation'] = org    # append org. data to the dataframe

top_listeners = top_listeners.reset_index().rename(columns = {'index':'IP Address',
                                                        'dst_IP':'No. of Packets'})
top_listeners

,No. of Packets,count,Organisation
0,103.37.198.100,3841,A-STAR-AS-AP
1,137.132.228.15,3715,NUSNET
2,202.21.159.244,2446,RPNET
3,192.101.107.153,2368,PNNL
4,103.21.126.2,2056,IITB-IN


## EXERCISE 4B: TRANSPORT PROTOCOL
Using the IP protocol type attribute, determine the percentage of TCP and UDP protocol

In [20]:
# Count occurrences of IP_protocol values and compute percentages
packet_counts = log_df['IP_protocol'].value_counts().to_frame(name='Count')
packet_percentages = log_df['IP_protocol'].value_counts(normalize=True).mul(100).to_frame(name='% total')

# Merge counts and percentages into a single DataFrame
packet_df = packet_counts.join(packet_percentages).reset_index()

# Rename the columns for clarity
packet_df.columns = ['Header Value', 'Count', '% total']

# Map IP_protocol values to known labels
protocol_mapping = {6: 'TCP', 17: 'UDP'}
packet_df['Protocol'] = packet_df['Header Value'].map(protocol_mapping)

# Filter only TCP and UDP packets
final_df = packet_df.dropna(subset=['Protocol'])[['Header Value', 'Count', '% total', 'Protocol']]

# Display the final DataFrame
final_df

,Header Value,Count,% total,Protocol
0,6,56064,80.818798,TCP
1,17,9462,13.639902,UDP


# EXERCISE 4C: APPLICATIONS PROTOCOL

### Using the Destination IP port number determine the most frequently used application protocol.
(For finding the service given the port number https://www.adminsub.net/tcp-udp-port-finder/ )


In [15]:
# Count occurrences of destination port values
dest_port_df = log_df['udp_dst_port/tcp_dst_port/icmp_code'].value_counts().nlargest(5).to_frame()

# Reset index and manually set column names
dest_port_df = dest_port_df.reset_index()
dest_port_df.columns = ['Destination Port', 'No. of Packets']

# Debugging step to verify the column names
print(dest_port_df.columns)  # Ensure 'Destination Port' exists before mapping

# Mapping ports to known services
port_mapping = {
    45512: 'Unassigned',
    443: 'HTTPS',
    80: 'HTTP',
    52866: 'Dynamic/Private Ports',
    56152: 'Dynamic/Private Ports',
    0: 'Reserved Port'
}

# Apply mapping with `.get()` instead of try-except
dest_port_df['Service'] = dest_port_df['Destination Port'].map(port_mapping).fillna('Unknown')

dest_port_df

Index(['Destination Port', 'No. of Packets'], dtype='object')


,Destination Port,No. of Packets,Service
0,443,13423,HTTPS
1,80,2647,HTTP
2,52866,2068,Dynamic/Private Ports
3,45512,1356,Unassigned
4,56152,1341,Dynamic/Private Ports


# EXERCISE 4D: TRAFFIC

### The traffic intensity is an important parameter that a network engineer needs to monitor closely to determine if there is congestion. You would use the IP packet size to calculate the estimated total traffic over the monitored period of 15 seconds. (Assume the sampling rate is 1 in 2048)  

In [16]:
total_traffic = sum(log_df['IP_size'])
# Assuming IP_size is in number of bits
total_traffic_Mb = total_traffic / (8 * pow(2, 20))
print(f"Total Traffic (Mb) = {total_traffic_Mb:.3f} Mb")

Total Traffic (Mb) = 7.722 Mb


# EXERCISE 4E: ADDITIONAL ANALYSIS

### Please append ONE page to provide additional analysis of the data and the insight it provides.
---

### Examples include:
### Top 5 communication pairs;
### Visualization of communications between different IP hosts; etc.
---
### Please limit your results within one page (and any additional results that fall beyond one page limit will not be assessed).

In [17]:
# Top 5 unique communication pairs
comm_pairs_df = log_df.groupby(['src_IP', 'dst_IP']).size().sort_values(ascending = False).to_frame()
comm_pairs_df.columns = ['No. of Packets']
comm_pairs_df = comm_pairs_df.reset_index()
# comm_pairs_df

top_comm_df = comm_pairs_df[:5]
top_comm_df

,src_IP,dst_IP,No. of Packets
0,193.62.192.8,137.132.228.15,3041
1,130.14.250.11,103.37.198.100,2599
2,14.139.196.58,192.101.107.153,2368
3,140.112.8.139,103.21.126.2,2056
4,137.132.228.15,193.62.192.8,1910


In [18]:
src_org = []
dst_org = []

for i in range(5):
    src_org.append(find_org(top_comm_df['src_IP'][i]))
    dst_org.append(find_org(top_comm_df['dst_IP'][i]))

top_comm_df['Source Organisation'] = src_org
top_comm_df['Destination Organisation'] = dst_org

top_comm_df = top_comm_df.reindex(['src_IP', 'Source Organisation', 'dst_IP',
                                   'Destination Organisation',
                                   'No. of Packets'], axis = 'columns')
top_comm_df

<ipython-input-18-ca03d8950391>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_comm_df['Source Organisation'] = src_org
<ipython-input-18-ca03d8950391>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_comm_df['Destination Organisation'] = dst_org


,src_IP,Source Organisation,dst_IP,Destination Organisation,No. of Packets
0,193.62.192.8,EUR-BIO-INST,137.132.228.15,NUSNET,3041
1,130.14.250.11,NLM-ETHER,103.37.198.100,A-STAR-AS-AP,2599
2,14.139.196.58,NKN-IIT-GUW,192.101.107.153,PNNL,2368
3,140.112.8.139,T-NTU.EDU.TW-NET,103.21.126.2,IITB-IN,2056
4,137.132.228.15,NUSNET,193.62.192.8,EUR-BIO-INST,1910
